In [ ]:
# |default_exp mongo_auteur

# Auteur

In [ ]:
# |export

from mongo import BaseEntity


class Auteur(BaseEntity):
    collection = "auteurs"

    def __init__(self, nom: str):
        """
        Auteur is a class that represents an author in the database auteurs.
        :param nom: The name of the author.
        """
        super().__init__(nom, self.collection)

In [ ]:
# liste les auteurs en base contenant le mot "marcel"
Auteur.get_entries("marcel")

[Marcel Proust]

In [ ]:
auteur1 = Auteur("Victor Hugo")
print(f"Est-ce que auteur1 existe ? {auteur1.exists()}")

auteur1.keep()
print(f"et maintenant, st-ce que auteur1 existe ? {auteur1.exists()}")
print(f"et voici l'id de auteur1 : {auteur1.get_oid()}")

auteur1.remove()
print(f"après nettoyage, est-ce que auteur1 existe ? {auteur1.exists()}")
print(f"et son oid : {auteur1.get_oid()}")

Est-ce que auteur1 existe ? False
et maintenant, st-ce que auteur1 existe ? True
et voici l'id de auteur1 : 67a73c579b0cd6b36ced0012
après nettoyage, est-ce que auteur1 existe ? False
et son oid : None


In [ ]:
from mongo import print_logs

print_logs(5)

{'_id': ObjectId('67a73c579b0cd6b36ced0014'), 'operation': 'delete', 'entite': 'auteurs', 'desc': 'Victor Hugo', 'date': datetime.datetime(2025, 2, 8, 12, 13, 27, 29000)}
{'_id': ObjectId('67a73c579b0cd6b36ced0011'), 'operation': 'insert', 'entite': 'auteurs', 'desc': 'Victor Hugo', 'date': datetime.datetime(2025, 2, 8, 12, 13, 27, 24000)}
{'_id': ObjectId('67a73a5da74538047917cf53'), 'operation': 'update', 'entite': 'auteurs', 'desc': 'Neige Sinno', 'date': datetime.datetime(2025, 2, 8, 12, 5, 1, 327000)}
{'_id': ObjectId('67a73a0e6de58ba219b27aab'), 'operation': 'delete', 'entite': 'auteurs', 'desc': 'Victor Hugo', 'date': datetime.datetime(2025, 2, 8, 12, 3, 42, 190000)}
{'_id': ObjectId('67a73a0e6de58ba219b27aa8'), 'operation': 'insert', 'entite': 'auteurs', 'desc': 'Victor Hugo', 'date': datetime.datetime(2025, 2, 8, 12, 3, 42, 184000)}


# AuthorChecker

In [ ]:
# | export

from thefuzz import fuzz
from thefuzz import process
from typing import List, Tuple

score_fuzz_threshold = 80


class AuthorFuzzMatcher:
    def __init__(self, reference_authors: List[str] = None):
        """Initialize with a list of known author names"""
        self.reference_authors = set(reference_authors) if reference_authors else set()

    def add_reference_author(self, author: str) -> None:
        """Add a new reference author to the set"""
        self.reference_authors.add(author.strip())

    def find_best_match(self, name: str, min_score: int = 80) -> Tuple[str, int]:
        """
        Find the best matching reference author for a given name
        Returns: (best_match, score)
        """
        if not name or not self.reference_authors:
            return None, 0
        # Find best match using token set ratio for better partial matching
        best_match, score = process.extractOne(
            name, self.reference_authors, scorer=fuzz.token_set_ratio
        )
        if score >= min_score:
            return best_match, score
        return None, score

In [ ]:
# | export

import os
from googleapiclient.discovery import build
from llm import load_env

load_env()
api_key = os.getenv("GOOGLE_CUSTOM_SEARCH_API_KEY")
cse_id = os.getenv("SEARCH_ENGINE_ID")

if not api_key or not cse_id:
    raise ValueError(
        "Les variables d'environnement GOOGLE_SEARCH_API_KEY et GOOGLE_CSE_ID doivent être définies."
    )


# Fonction de recherche Google
def google_search(query):
    try:
        service = build("customsearch", "v1", developerKey=api_key)
        res = service.cse().list(q=query, cx=cse_id).execute()

        results = []
        for item in res.get("items", []):
            title = item.get("title")
            snippet = item.get("snippet")
            link = item.get("link")
            results.append({"title": title, "snippet": snippet, "link": link})
        return results
    except Exception as e:
        print(f"Erreur lors de la recherche Google: {e}")
        return None

In [ ]:
# | export

from mongo_auteur import Auteur
from mongo_episode import Episode
from llm import get_azure_llm
from llama_index.core.llms import ChatMessage
import json


class AuthorChecker:

    def __init__(self, episode: Episode):
        self.episode = episode
        self.llm_structured_output = get_azure_llm("gpt-4o")
        self.authors_titre_description = self._get_authors_from_titre_description()

    def _get_authors_from_titre_description(self):
        response_schema = {
            "type": "json_schema",
            "json_schema": {
                "name": "AuthorTitreDescriptionList",
                "schema": {
                    "type": "object",
                    "properties": {
                        "Authors_TitreDescription": {
                            "type": "array",
                            "items": {
                                "type": "string",
                                "description": "A list of names from title and description",
                            },
                        }
                    },
                    "required": ["Authors_TitreDescription"],
                    "additionalProperties": False,
                },
            },
        }
        response = self.llm_structured_output.chat(
            messages=[
                ChatMessage(
                    role="system",
                    content="Tu es un assistant utile qui retourne une liste JSON de noms.",
                ),
                ChatMessage(
                    role="user",
                    content=f"Est-ce que tu peux me lister tous les noms \
                            qui sont cités dans le titre et la description de l'épisode suivant : \
                            {self.episode.titre} {self.episode.description}. ",
                ),
            ],
            response_format=response_schema,
        )
        try:
            json_dict = json.loads(response.message.content)
        except json.JSONDecodeError as e:
            print("Error parsing JSON:", e)
            print("Raw response:", json_dict)
        return json_dict["Authors_TitreDescription"]

    def _get_authors_from_llm(self, autor) -> List[str]:

        response_schema = {
            "type": "json_schema",
            "json_schema": {
                "name": "AuthorList",
                "schema": {
                    "type": "object",
                    "properties": {
                        "Authors_LLM": {
                            "type": "array",
                            "items": {
                                "type": "string",
                                "description": "A list of authors' names",
                            },
                        }
                    },
                    "required": ["Authors_LLM"],
                    "additionalProperties": False,
                },
            },
        }

        prompt = (
            """
        Tu es un agent expert en littérature.
        Donne moi quelques auteurs dont le nom s'approche de celui-ci : """
            + autor
            + """

        S'il s'agit deja d'un auteur connu, retourne moi juste son nom. S'il y a une erreur dans le nom que je t'ai donne, corrige moi en me donnant le nom de l'auteur que tu penses que j'ai voulu dire.

        Je veux que tu me donnes le prenom puis le nom dans cet ordre. Par exemple "Marcel Pagnol" ou "Victor Hugo".
        Ces auteurs sont susceptibles d'etre discutes dans "Le Masque et la Plume".

        Si tu me retournes plusieurs auteurs, fais le sous forme de liste par exemple si tu as identifie "auteur 1" et "auteur 2" alors retourne ["auteur 1", "auteur 2"]
        """
        )

        response = self.llm_structured_output.chat(
            messages=[
                ChatMessage(
                    role="system",
                    content="Tu es un agent litteraire qui connait parfaitement les auteurs.",
                ),
                # ChatMessage(role="user", content=f"{query}. Please provide the response in JSON format as a list of strings, following this schema: ['author1', 'author2', ...]")
                ChatMessage(role="user", content=f"{prompt}. "),
            ],
            response_format=response_schema,
        )

        try:
            json_dict = json.loads(response.message.content)
            # ...
        except json.JSONDecodeError as e:
            print("Error parsing JSON:", e)
            print("Raw response:", response)
        return json_dict["Authors_LLM"]

    def _get_author_from_web(self, author: str):

        result_google = google_search(author)

        prompt_incertitude_auteur = f"""
        Voici le resultat d'une requete google concernant un probable auteur inconnu de mon llm : {author}
        La requete est au format dict avec du json a l'interieur.
        Est-ce que tu peux analyser le contenu de cette requete et me dire si oui ou non {author} est un auteur de livres, 
        et accompagner ta reponse d'un pourcentage de certitude :
        * 100% de certitude signifie que tu es certain que {author} est un auteur de livres
        *  50% tu es ni sure ni pas sure que {author} est un auteur de livres
        *   0% tu es certain que {author} n'est pas un auteur de livres

        Voici le contenu de la requete google : {result_google}

        Tu repondras uniquement avec un dictionnaire qui va contenir 3 entrees :

        - "auteur" : le nom de l'auteur, eventuellement corrige si j'ai oublie des accents ou une faute de frappe
        - "certitude" : le pourcentage de certitude de 0 à 100, un entier
        - "analyse" : une analyse de la requete google
        """
        response_schema = {
            "type": "json_schema",
            "json_schema": {
                "name": "AuteurSchema",
                "schema": {
                    "type": "object",
                    "properties": {
                        "auteur": {
                            "type": "string",
                            "description": "Le nom de l'auteur éventuellement corrigé (accents, fautes de frappe).",
                        },
                        "certitude": {
                            "type": "integer",
                            "description": "Pourcentage de certitude (0 à 100).",
                            "minimum": 0,
                            "maximum": 100,
                        },
                        "analyse": {
                            "type": "string",
                            "description": "Analyse de la requête Google concernant l'auteur.",
                        },
                    },
                    "required": ["auteur", "certitude", "analyse"],
                    "additionalProperties": False,
                },
            },
        }

        response = self.llm_structured_output.chat(
            messages=[
                ChatMessage(
                    role="system",
                    content="Tu es un assistant utile qui analyse des requetes Google pour y deceler si un auteur de livre s'y cache.",
                ),
                ChatMessage(
                    role="user",
                    content=prompt_incertitude_auteur,
                ),
            ],
            response_format=response_schema,
        )
        try:
            json_dict = json.loads(response.message.content)
        except json.JSONDecodeError as e:
            print("Error parsing JSON:", e)
            print("Raw response:", json_dict)
        return json_dict

    def _check_author_source(self, author: str, authors_list: list[str]) -> str | None:
        """Essaie de faire correspondre 'author' dans 'authors_list' et retourne la meilleure correspondance ou None."""
        matcher = AuthorFuzzMatcher(authors_list)
        best_match, score = matcher.find_best_match(author)
        if score >= score_fuzz_threshold:
            return best_match
        else:
            return None

    def check_author(
        self, author: str, return_details: bool = False, verbose: bool = False
    ) -> str | dict | None:
        """
        Vérifie l'auteur en testant différentes sources et retourne soit :
          - une chaîne (le nom corrigé) si return_details=False,
          - ou un dictionnaire détaillé avec plusieurs informations si return_details=True.

        Les étapes testées sont :
          - rss:metadata (titre, description)
          - mongodb:auteurs (déjà connus)
          - llm
          - web search
        """
        details = {"author_original": author, "author_corrected": None, "source": None}

        # 1. Vérification dans rss:metadata (titre, description)
        match = self._check_author_source(author, self.authors_titre_description)
        if match:
            details["author_corrected"] = match
            details["source"] = "rss:metadata"
            if verbose:
                print(f"Trouvé avec rss:metadata: {match}")
            return details if return_details else match

        # 2. Vérification dans la base de données (mongodb:auteurs)
        list_db_auteurs = [auteur.nom for auteur in Auteur.get_entries()]
        match = self._check_author_source(author, list_db_auteurs)
        if match:
            details["author_corrected"] = match
            details["source"] = "mongodb:auteurs"
            if verbose:
                print(f"Trouvé avec mongodb:auteurs: {match}")
            return details if return_details else match

        # 3. Vérification via llm
        list_llm_auteurs = self._get_authors_from_llm(author)
        match = self._check_author_source(author, list_llm_auteurs)
        if match:
            details["author_corrected"] = match
            details["source"] = "llm"
            if verbose:
                print(f"Trouvé avec llm: {match}")
            return details if return_details else match

        # 4. Vérification via web search
        web_result_dict = self._get_author_from_web(author)
        match = web_result_dict.get("auteur")
        score = web_result_dict.get("certitude", 0)
        # Ajout d'infos web dans le dictionnaire détaillé
        details.update(
            {
                "author_corrected": match,
                "score": score,
                "analyse": web_result_dict.get("analyse", ""),
                "source": "web search",
            }
        )
        if score >= score_fuzz_threshold:
            if verbose:
                print(f"Trouvé avec web search: {match}")
            return details if return_details else match
        else:
            if verbose:
                print(
                    f"Score insuffisant {score} avec web search: {web_result_dict.get('analyse', '')}"
                )
                # Ajout d'infos web dans le dictionnaire détaillé
            details.update(
                {
                    "author_corrected": None,
                }
            )
            return details if return_details else None

In [ ]:
import datetime
from mongo_episode import Episode

episode_date = datetime.date(2025, 1, 12)
ac = AuthorChecker(Episode.from_date(episode_date))
ac.check_author("Jeanne Riviere", verbose=True)

Trouvé avec mongodb:auteurs: Jeanne Rivière


'Jeanne Rivière'

In [ ]:
ac.check_author("Jeanne Riviere", return_details=True)

{'author_original': 'Jeanne Riviere',
 'author_corrected': 'Jeanne Rivière',
 'source': 'mongodb:auteurs'}

In [ ]:
ac.check_author("Neige Sinault", return_details=True, verbose=True)

Score insuffisant 0 avec web search: La requête Google ne contient aucune information indiquant que Neige Sinault est un auteur de livres. Les résultats mentionnent principalement Philippe Sinault, qui est lié au sport automobile et à l'équipe Signatech-Alpine. Aucun des extraits ne fait référence à Neige Sinault en tant qu'auteur ou à des œuvres littéraires associées à ce nom. Par conséquent, il est très peu probable que Neige Sinault soit un auteur de livres.


{'author_original': 'Neige Sinault',
 'author_corrected': None,
 'source': 'web search',
 'score': 0,
 'analyse': "La requête Google ne contient aucune information indiquant que Neige Sinault est un auteur de livres. Les résultats mentionnent principalement Philippe Sinault, qui est lié au sport automobile et à l'équipe Signatech-Alpine. Aucun des extraits ne fait référence à Neige Sinault en tant qu'auteur ou à des œuvres littéraires associées à ce nom. Par conséquent, il est très peu probable que Neige Sinault soit un auteur de livres."}

# extract py

In [ ]:
from nbdev.export import nb_export

nb_export("py mongo helper auteurs.ipynb", ".")